In [1]:
#1.dependent feature and independent variable must be numeric
#2.if not numeric then convert to numeric valuesx

import findspark
findspark.init()

In [3]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("HTwtApp").getOrCreate()

In [52]:
df=spark.read.csv("F:\\datasets\\weight-height.csv",header=True,inferSchema=True)
df.show(5,False)

+------+----------------+----------------+
|Gender|Height          |Weight          |
+------+----------------+----------------+
|Male  |73.847017017515 |241.893563180437|
|Male  |68.7819040458903|162.3104725213  |
|Male  |74.1101053917849|212.7408555565  |
|Male  |71.7309784033377|220.042470303077|
|Male  |69.8817958611153|206.349800623871|
+------+----------------+----------------+
only showing top 5 rows



In [14]:
df.printSchema()
df.count()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)



10000

In [16]:
df.na.drop()
df.count()

10000

In [19]:
#formatting of data
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['Height'],outputCol='HeightData')
dff=featureassembler.transform(df)
dff.show()


+------+----------------+----------------+------------------+
|Gender|          Height|          Weight|        HeightData|
+------+----------------+----------------+------------------+
|  Male| 73.847017017515|241.893563180437| [73.847017017515]|
|  Male|68.7819040458903|  162.3104725213|[68.7819040458903]|
|  Male|74.1101053917849|  212.7408555565|[74.1101053917849]|
|  Male|71.7309784033377|220.042470303077|[71.7309784033377]|
|  Male|69.8817958611153|206.349800623871|[69.8817958611153]|
|  Male|67.2530156878065|152.212155757083|[67.2530156878065]|
|  Male|68.7850812516616|183.927888604031|[68.7850812516616]|
|  Male|68.3485155115879|167.971110489509|[68.3485155115879]|
|  Male| 67.018949662883| 175.92944039571| [67.018949662883]|
|  Male|63.4564939783664|156.399676387112|[63.4564939783664]|
|  Male|71.1953822829745|186.604925560358|[71.1953822829745]|
|  Male|71.6408051192206|213.741169489411|[71.6408051192206]|
|  Male|64.7663291334055|167.127461073476|[64.7663291334055]|
|  Male|

In [21]:
dff.select('HeightData','Weight').show()

+------------------+----------------+
|        HeightData|          Weight|
+------------------+----------------+
| [73.847017017515]|241.893563180437|
|[68.7819040458903]|  162.3104725213|
|[74.1101053917849]|  212.7408555565|
|[71.7309784033377]|220.042470303077|
|[69.8817958611153]|206.349800623871|
|[67.2530156878065]|152.212155757083|
|[68.7850812516616]|183.927888604031|
|[68.3485155115879]|167.971110489509|
| [67.018949662883]| 175.92944039571|
|[63.4564939783664]|156.399676387112|
|[71.1953822829745]|186.604925560358|
|[71.6408051192206]|213.741169489411|
|[64.7663291334055]|167.127461073476|
|[69.2830700967204]|189.446181386738|
|[69.2437322298112]|186.434168021239|
|[67.6456197004212]|172.186930058117|
|[72.4183166259878]|196.028506330482|
| [63.974325721061]| 172.88347020878|
|[69.6400598997523]| 185.98395757313|
|[67.9360048540095]|182.426648013226|
+------------------+----------------+
only showing top 20 rows



In [22]:
dff.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Height: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- HeightData: vector (nullable = true)



In [23]:
fd=dff.select('HeightData','Weight')
fd.show()
#preprocessing of data

+------------------+----------------+
|        HeightData|          Weight|
+------------------+----------------+
| [73.847017017515]|241.893563180437|
|[68.7819040458903]|  162.3104725213|
|[74.1101053917849]|  212.7408555565|
|[71.7309784033377]|220.042470303077|
|[69.8817958611153]|206.349800623871|
|[67.2530156878065]|152.212155757083|
|[68.7850812516616]|183.927888604031|
|[68.3485155115879]|167.971110489509|
| [67.018949662883]| 175.92944039571|
|[63.4564939783664]|156.399676387112|
|[71.1953822829745]|186.604925560358|
|[71.6408051192206]|213.741169489411|
|[64.7663291334055]|167.127461073476|
|[69.2830700967204]|189.446181386738|
|[69.2437322298112]|186.434168021239|
|[67.6456197004212]|172.186930058117|
|[72.4183166259878]|196.028506330482|
| [63.974325721061]| 172.88347020878|
|[69.6400598997523]| 185.98395757313|
|[67.9360048540095]|182.426648013226|
+------------------+----------------+
only showing top 20 rows



In [24]:
#spliting of data into train and test data
from pyspark.ml.regression import LinearRegression

In [25]:
train,test=fd.randomSplit([.70,.30])

In [31]:
lr=LinearRegression(featuresCol='HeightData',labelCol='Weight')
tm=lr.fit(train)

In [36]:

#coefficient and intercept
print(tm.intercept)#intercept
print(tm.coefficients)#coefficient  bet weight and height


-353.62677174290985
[7.760006146833987]


In [40]:
#evaluate the train data
res=tm.evaluate(test)
res.predictions.show()

+------------------+----------------+-----------------+
|        HeightData|          Weight|       prediction|
+------------------+----------------+-----------------+
|[54.2631333250971]| 64.700126712753|67.45547640631582|
|[54.6168578301035]| 71.393748738973| 70.2003807394513|
|[55.1485573624105]| 88.812412112758|74.32637237842238|
|[55.6682021205121]|68.9825300912419|78.35881889546084|
| [56.098245784398]|104.954100411731|81.69596037062246|
|[56.1053695899164]| 87.298869133415|81.75124114523408|
|[56.5479749809036]|84.8721236452797|85.18586169991659|
|[56.5488430793485]|90.8475893808927|85.19259814918507|
|[56.6304119764382]| 89.480480272958|85.82557329199159|
|[56.7576036320284]|88.8848531761247|86.81258132119757|
|[56.8103172829116]|84.1706947685606|87.22163957607324|
|[56.8560821293767]|97.3649783271705|87.57677506595138|
|[56.9752789632804]|90.3417842608302|88.50174322972725|
|[57.0726562519626]|93.7461421572008|89.25739158846312|
|[57.1038694679138]| 93.506315903823|89.49960633

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\sql\context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [44]:
print("this is R2 ",res.r2)#--accuracy
print("this is mean absolute error is",res.meanAbsoluteError)
print("Root mean square error(RMSE)",res.rootMeanSquaredError)

this is R2  0.8577618544734198
this is mean absolute error is 9.631205326953106
Root mean square error(RMSE) 12.093136204198746


In [50]:
#saving ,loading ,and using new model
#tm.save("e:\\myperfectmodel")
#mynewmod=tm.load("e:\\myperfectmodel")
#mynewmod.evaluate(test)